In [33]:
%config Completer.use_jedi = False
import matplotlib
matplotlib.use('TkAgg')
from pyquaternion import Quaternion

In [59]:
import os
import numpy as np
import pandas as pd
from operator import truediv, mul
import array
from matplotlib import rcParams
import matplotlib.pyplot as plt
from matplotlib import lines

matplotlib.rcdefaults()
matplotlib.rcParams['pdf.fonttype']=42
matplotlib.rcParams['ps.fonttype']=42
font = {
        'weight': 'normal',
        'size': 13,
            }
matplotlib.rcParams['legend.framealpha']=0.8
matplotlib.rc('font', **font) 


In [60]:
import seaborn as sns

In [61]:
def resampling(map_data):
    v = []
    area = []
    ent = []
    cnt = 0
    for i in map_data.item()['time']:
        v.append(float(i.to_sec()))
        if(map_data.item()['area'][cnt] != 0):
            area.append(map_data.item()['area'][cnt])
        else:
            area.append(0.0025)
        ent.append(map_data.item()['total_entropy'][cnt])
        cnt += 1
    d = pd.DataFrame(columns=['ts','area','ent'])

    d.loc[:,'ts'] = v
    d.loc[:,'area'] = area
    d.loc[:,'ent'] = ent
    d = d.set_index('ts')
    d.index = pd.to_datetime(d.index, unit='s')
    tmp = d.resample('2S')[['area']].agg(lambda x : np.nan if x.count() == 0 else x.idxmax()).ffill()
    df = d.loc[tmp['area']]
    cnt = 0

    while (len(df) < 150):
        new_row = pd.DataFrame(columns=['area','ent'])
        new_row.loc[0 + cnt / 1000.0,'area'] = 0.0025
        new_row.loc[0 + cnt / 1000.0,'ent'] = 1
        cnt += 1
        df = pd.concat([new_row, df])
        
    return df

In [62]:
def get_rmse(occupancy, gt):
    oc = []
    se = 0
    for l in occupancy:
        p = float(l)
        oc.append(p)
    oc_gt = []
    for l in gt:
        p = float(l)
        oc_gt.append(p)
    for l in range(len(oc)):
        if(oc[l] == -1):
            oc[l] = 50
        if(oc_gt[l] == -1):
            oc_gt[l] = 50
        se += (oc[l] - oc_gt[l])**2
    mse = se/len(oc)
    return mse**0.5

def get_rmse_known(occupancy, gt):
    oc = []
    se = 0
    for l in occupancy:
        p = float(l)
        oc.append(p)
    oc_gt = []
    for l in gt:
        p = float(l)
        oc_gt.append(p)
    cnt = 0.0
    for l in range(len(oc)):
        if (oc[l] == -1):
            if (oc_gt[l] != -1):
                oc[l] = 50
                cnt += 1
                se += (oc[l] - oc_gt[l])**2
        else:
            if(oc_gt == -1):
                oc_gt[l] = 100 - oc[l]
            cnt += 1
            se += (oc[l] - oc_gt[l])**2
    mse = se/cnt
    return mse**0.5

def bac_manual(occupancy, gt):
    oc = []
    se = 0
    for l in occupancy:
        p = float(l)
        oc.append(p)
    oc_gt = []
    for l in gt:
        p = float(l)
        oc_gt.append(p)
    tot_free = 0
    tot_occ = 0
    corr_free = 0
    corr_occ = 0
    tot_unk = 0
    corr_unk = 0
    for l in range(len(oc)):
        if oc_gt[l] == 0:
            tot_free += 1
            if oc[l] == oc_gt[l]:
                corr_free += 1
        elif oc_gt[l] == 100:
            tot_occ += 1
            if oc[l] == oc_gt[l]:
                corr_occ += 1
        elif oc_gt[l] == -1:
            tot_unk += 1
            if oc[l] == oc_gt[l]:
                corr_unk += 1
    return [1/3.0*(corr_free/tot_free + corr_occ/tot_occ + corr_unk/tot_unk),corr_free/tot_free,corr_occ/tot_occ, corr_unk/tot_unk]


In [63]:
#######################################################################################################################
#Author   : Dr. Arun B Ayyar
#
#Based on : Shimazaki H. and Shinomoto S., A method for selecting the bin size of a time histogram Neural Computation (2007)
#	   Vol. 19(6), 1503-1527
#
#Data     : The duration for eruptions of the Old Faithful geyser in Yellowstone National Park (in minutes) 
#	   or normal distribuition.
#	   given at http://176.32.89.45/~hideaki/res/histogram.html
#
#Comments : Implements a faster version than using hist from matplotlib and histogram from numpy libraries	
#           Also implements the shifts for the bin edges
#
########################################################################################################################


def compute_bin_size(data):
    data_max = max(data) #lower end of data
    data_min = min(data) #upper end of data
    n_min = 2500   #Minimum number of bins Ideal value = 2
    n_max = 2501  #Maximum number of bins  Ideal value =200
    n_shift = 1     #number of shifts Ideal value = 30
    N = np.array(range(n_min,n_max))
    D = float(data_max-data_min)/N    #Bin width vector
    Cs = np.zeros((len(D),n_shift)) #Cost function vector
    #Computation of the cost function
    for i in range(np.size(N)):
        shift = np.linspace(0,D[i],n_shift)
        for j in range(n_shift):
            edges = np.linspace(data_min+shift[j]-D[i]/2,data_max+shift[j]-D[i]/2,N[i]+1) # shift the Bin edges
            binindex = np.digitize(data,edges) #Find binindex of each data point
            ki= np.bincount(binindex)[1:N[i]+1] #Find number of points in each bin
            k = np.mean(ki) #Mean of event count
            v = sum((ki-k)**2)/N[i] #Variance of event count
            Cs[i,j]+= (2*k-v)/((D[i])**2) #The cost Function
    C=Cs.mean(1)

    #Optimal Bin Size Selection
    loc = np.argwhere(Cs==Cs.min())[0]
    cmin = C.min()
    idx  = np.where(C==cmin)
    idx = idx[0][0]
    optD = D[idx]
    return np.linspace(data_min+shift[loc[1]]-D[idx]/2,data_max+shift[loc[1]]-D[idx]/2,N[idx]+1,endpoint=False)

In [69]:
def get_data(folder, columns, kind, df, low = 0, r = 10, ate = False, saving_results = False):
    color = ['tab:blue','tab:green','tab:red','tab:orange','tab:purple','tab:brown','tab:pink','tab:gray','tab:olive','tab:cyan','k']
    final_folders = []

    for i in columns:
        for k in kind:
            final_folders.append(os.path.join(folder, i+k))

    cnt = 0
    fig1, (ax1, ax2) = plt.subplots(1,2)    

    fig_map, (ax_map) = plt.subplots(1)
    fig_exp, (ax_exp) = plt.subplots(1)
    font = {
        'weight': 'normal',
        'size': 18,
            }
    ax_exp.set_ylabel('explored %', fontdict=font)
    ax_exp.set_xlabel('trajectory length [m]', fontdict=font)
    ax_map.set_ylabel('explored %', fontdict=font)
    ax_map.set_xlabel('time [s]', fontdict=font)

    ax_map_split = ax_map.twinx()
    ax_map_split.set_ylabel('trajectory length [m]', fontdict=font)
    ax_map_split.set_ylim([0,120])
    sec_y = ax_exp.secondary_yaxis('right')
    sec_y.set_ylabel('normalized entropy', fontdict=font)

    for i in final_folders:
        tot_score = []
        tot_BAC =  []
        tot_BAC_free = []
        tot_BAC_occ = []
        cam_yaw_total = []
        rob_yaw_total = []
        if ate:
            tot_TRPE = []
            tot_RRPE = []
            tot_ATE = []
        cov_vec = []
        tot_lc = []
        glob_lc = []
        loc_lc = []
        tot_area = []
        area_bucketed = []
        entropy_bucketed = []
        pose_cov_vec = []
        final_map_rmse = []
        bac_manually = []
        bac_manually_unk = []
        bac_manually_free = []
        bac_manually_occ = []
        path_len =[]
        filtered_path_df = []
        wheel_tot = []
        for j in range(low,r):
            current = os.path.join(i,str(j))
            print(current)
            if (os.path.isfile(os.path.join(current,'map_data.npy'))):
                if(os.path.isfile(os.path.join(current,'rposes.txt'))):
                    raw_file = open(os.path.join(current,'rposes.txt'),'r')
                    Lines = raw_file.readlines()
                    tot_yaw_r = 0.0
                    first_yaw = True
                    prev_yaw = 0
                    current_yaw_robot = []
                    for l in Lines:
                        line = l.split(' ')
                        q = Quaternion(float(line[-1]),float(line[-4]),float(line[-3]),float(line[-2]))
                        if first_yaw:
                            first_yaw = False
                            or_base = q
                            prev_yaw = (q/or_base).yaw_pitch_roll[0]
                            current_yaw_robot.append(prev_yaw)
                        else:
                            anglediff = ((q/or_base).yaw_pitch_roll[0] - prev_yaw + np.pi + np.pi*2) % (np.pi*2) - np.pi
                            tot_yaw_r += abs(anglediff)
                            prev_yaw = (q/or_base).yaw_pitch_roll[0]
                            current_yaw_robot.append(prev_yaw)

                    first_yaw = True

                    raw_file = open(os.path.join(current,'cposes.txt'),'r')
                    Lines = raw_file.readlines()
                    tot_yaw_c = 0
                    count_angles = 0
                    for l in Lines:
                        line = l.split(' ')
                        q = Quaternion(float(line[-1]),float(line[-4]),float(line[-3]),float(line[-2]))
                        if first_yaw:
                            first_yaw = False
                            cam_base = q
                            yaw_wrt_r = ((q/cam_base).yaw_pitch_roll[0] - current_yaw_robot[count_angles]+ np.pi + np.pi*2) % (np.pi*2) - np.pi
                            prev_yaw = yaw_wrt_r
                        else:
                            yaw_wrt_r = ((q/cam_base).yaw_pitch_roll[0] - current_yaw_robot[count_angles]+ np.pi + np.pi*2) % (np.pi*2) - np.pi
                            anglediff = (prev_yaw - yaw_wrt_r + np.pi + np.pi*2) % (np.pi*2) - np.pi
                            tot_yaw_c += abs(anglediff)
                            prev_yaw = yaw_wrt_r
                        count_angles += 1
                        
                    cam_yaw_total.append(tot_yaw_c)
                    rob_yaw_total.append(tot_yaw_r)
                map_data = np.load(os.path.join(current,'map_data.npy'), allow_pickle=True, encoding="latin1")
                system_data = np.load(os.path.join(current,'feat.npy'), allow_pickle=True, encoding="latin1")
#                 wheel_data = np.load(os.path.join(current,'wheel_from_odom.npy'), allow_pickle=True, encoding="latin1")
#                 poses = open(os.path.join(current,'poses.g2o'), 'r')
                tot_lc.append(system_data.item()['total'][-1])
                glob_lc.append(system_data.item()['global'][-1])
                loc_lc.append(system_data.item()['local'][-1])
                tot_area.append(map_data.item()['area'][-1])
#                 wheel_tot.append(wheel_data.item()['tot_wheel'][-1])
                wheel_found = False
                
                resampled = resampling(map_data)
                area_bucketed.append(resampled[-150:].loc[:,'area'])            
                entropy_sliced = resampled[-150:].loc[:,'ent']       

                a = [0.0025 for i in range(len(area_bucketed[-1]))]
                res = list(map(truediv, area_bucketed[-1], a))
                res2 = list(map(truediv, entropy_sliced, res))
                entropy_bucketed.append(res2)

                path_gt = open(os.path.join(current,'rtabmap_odom.txt'),'r')
                prev = [0,0,0]
                first = True
                second = True
                path_df = pd.DataFrame(columns=['time','len'])
                lt = 0
                init_time = 0
                for l in path_gt:
                    p = l.split()
                    if first:
                        prev = [float(p[1]), float(p[2])]
                        path_df.loc[p[0],'time'] = float(p[0])
                        path_df.loc[p[0],'len'] = 0
                        init_time =float(p[0])
                        cum_dist = 0
                        first = False
                    else:
                        if second and float(p[0])-init_time > 1:
                            init_time = float(p[0])
                        else:
                            second = False
                        cum_dist += ((prev[0]-float(p[1]))**2+(prev[1]-float(p[2]))**2)**0.5
                        path_df.loc[p[0],'time'] = float(p[0])
                        path_df.loc[p[0],'len'] = cum_dist
                        prev = [float(p[1]), float(p[2])]
                        lt = float(p[0])
#                 for wl in range(len(wheel_data.item()['time'])):
#                     if wheel_data.item()['time'][wl].to_sec() >= init_time:
#                         for wl_second in range(wl+1, len(wheel_data.item()['time'])):
#                             if wheel_data.item()['time'][wl_second].to_sec() - wheel_data.item()['time'][wl].to_sec() >= 299 or wheel_data.item()['time'][wl_second].to_sec() >= lt:
#                                 if (wheel_data.item()['time'][wl_second].to_sec() - wheel_data.item()['time'][wl].to_sec() > 300):
#                                     wheel_tot[-1] = abs(wheel_data.item()['tot_wheel'][wl_second] - wheel_data.item()['tot_wheel'][wl])
#                                 else:
#                                     wheel_tot[-1] = abs(wheel_data.item()['tot_wheel'][wl_second] - wheel_data.item()['tot_wheel'][wl])
#                                 break
#                         print('----')
#                         break


                path_len.append(cum_dist)
                path_df.index = pd.to_datetime(path_df.index, unit='s')
                tmp_filtered_path_df = path_df.resample('2S').max().ffill()
                norm = 0
                while (len(tmp_filtered_path_df) < 150):

                    new_row = pd.DataFrame(columns=['time','len'])
                    new_row.loc[0,'time'] = 0 + norm/1000.0
                    norm += 1
                    new_row.loc[0,'len'] = 0.00001
                    new_row = new_row.set_index('time')
                    new_row.index = pd.to_datetime(new_row.index, unit='s')
                    tmp_filtered_path_df = pd.concat([new_row, tmp_filtered_path_df])

                filtered_path_df.append(tmp_filtered_path_df[-150:]['len'])

                # collect link variances of the graph
#                 for l in poses:
#                     if 'EDGE' in l:
#                         s = l.split()
#                         m = np.array([[float(s[-6]), float(s[-5]), float(s[-4])],
#                                       [float(s[-5]), float(s[-3]), float(s[-2])],
#                                       [float(s[-4]), float(s[-2]), float(s[-1])]]
#                                     )
#                         cov = np.linalg.inv(m)
#                         eigen = np.linalg.eig(cov)
#                         cov_vec.append(np.exp(1/3.0 * np.sum(np.log(eigen[0]))).real)
                        #cov_vec.append(np.linalg.det(cov))
        if(len(path_len)>0):
#             wheel_over_path = list(map(truediv, wheel_tot, path_len))
#             print(wheel_over_path)
#             ry_over_path = list(map(truediv, rob_yaw_total, path_len))
#             cy_over_path = list(map(truediv, cam_yaw_total, path_len))
#             df.loc['tot_lc',i.split('/')[-1]] =  'mean: ' + str(np.mean(tot_lc)) + ' ' + ' std: ' + str(np.std(tot_lc))
            df.loc['tot_area',i.split('/')[-1]] =  'mean: ' + str(np.mean(tot_area)) + ' ' + ' std: ' + str(np.std(tot_area))
#             df.loc['pl',i.split('/')[-1]] =  'mean: ' + str(np.mean(path_len)) + ' ' + ' std: ' + str(np.std(path_len))
#             df.loc['tot_y_c',i.split('/')[-1]] =  'mean: ' + str(np.mean(cam_yaw_total)) + ' ' + ' std: ' + str(np.std(cam_yaw_total))
#             df.loc['tot_y_r',i.split('/')[-1]] =  'mean: ' + str(np.mean(rob_yaw_total)) + ' ' + ' std: ' + str(np.std(rob_yaw_total))
#             df.loc['tot_wheel',i.split('/')[-1]] =  'mean: ' + str(np.mean(wheel_tot)) + ' ' + ' std: ' + str(np.std(wheel_tot))
#             df.loc['final_entropy_norm',i.split('/')[-1]] =  'mean: {:.2f} std: {:.2f}'.format(round(np.mean(entropy_bucketed, axis = 0)[-1],2),round(np.std(entropy_bucketed, axis = 0)[-1],2))
#             df.loc['final_rmse',i.split('/')[-1]] =  'mean: ' + str(np.mean(final_map_rmse)) + ' ' + ' std: ' + str(np.std(final_map_rmse))
            if ate:
                if (saving_results):
                    df.loc['tot_area',i.split('/')[-1]] =  'mean: ${:.3f}$ std: ${:.3f}$'.format(round(np.mean(tot_area, axis = 0),3),round(np.std(tot_area, axis = 0),3))
                    df.loc['final_entropy_norm',i.split('/')[-1]] =  'mean: ${:.3f}$ std: ${:.3f}$'.format(round(np.mean(entropy_bucketed, axis = 0)[-1],3),round(np.std(entropy_bucketed, axis = 0)[-1],3))
#                     df.loc['per_m_wheel',i.split('/')[-1]] =  'mean: ${:.3f}$ std: ${:.3f}$'.format(round(np.mean(wheel_over_path),3), + round(np.std(wheel_over_path),3))
#                     df.loc['per_m_rob',i.split('/')[-1]] =  'mean: ${:.3f}$ std: ${:.3f}$'.format(round(np.mean(ry_over_path),3), + round(np.std(ry_over_path),3))
#                     df.loc['per_m_cam',i.split('/')[-1]] =  'mean: ${:.3f}$ std: ${:.3f}$'.format(round(np.mean(cy_over_path),3), + round(np.std(cy_over_path),3))
                    df.loc['path length',i.split('/')[-1]] =  'mean: ${:.3f}$ std: ${:.3f}$'.format(round(np.mean(path_len),3), + round(np.std(path_len),3))
#                 else:
#                     df.loc['per_m_wheel',i.split('/')[-1]] =  'mean: {:.3f} std: {:.3f}'.format(round(np.mean(wheel_over_path),3), + round(np.std(wheel_over_path),3))
#                     df.loc['per_m_rob',i.split('/')[-1]] =  'mean: {:.3f} std: {:.3f}'.format(round(np.mean(ry_over_path),3), + round(np.std(ry_over_path),3))
#                     df.loc['per_m_cam',i.split('/')[-1]] =  'mean: {:.3f} std: {:.3f}'.format(round(np.mean(cy_over_path),3), + round(np.std(cy_over_path),3))

#                 df.loc['tot_TRPe',i.split('/')[-1]] =  'mean: ' + str(np.mean(tot_TRPE)) + ' ' + ' std: ' + str(np.std(tot_TRPE))
#                 df.loc['tot_RRPe',i.split('/')[-1]] =  'mean: ' + str(np.mean(tot_RRPE)) + ' ' + ' std: ' + str(np.std(tot_RRPE))
#                 df.loc['path_len',i.split('/')[-1]] =  'mean: ' + str(np.mean(path_len)) + ' ' + ' std: ' + str(np.std(path_len))
#                 df.loc['area over path',i.split('/')[-1]] =  'mean: ' + str(np.mean(tot_area)/np.mean(path_len))
                lc_over_path = list(map(truediv, tot_lc, path_len))
                print(lc_over_path)
    
                if (saving_results):
                    df.loc['Loops per m',i.split('/')[-1]] =  'mean: ${:.3f}$ std: ${:.3f}$'.format(round(np.mean(lc_over_path),3),round(np.std(lc_over_path),3))
                else:
                    df.loc['Loops per m',i.split('/')[-1]] =  'mean: {:.2f} std: {:.2f}'.format(round(np.mean(lc_over_path),3),round(np.std(lc_over_path),3))
                
            panda = pd.DataFrame(columns=['time','norm_entropy','area','path_len','exp'])
            cnt_exp = 0
            for tries in range(len(entropy_bucketed)):
                for result in range(len(entropy_bucketed[tries])):
                    panda.loc[cnt_exp,'norm_entropy'] = entropy_bucketed[tries][result]
                    panda.loc[cnt_exp,'time'] = result*2
                    panda.loc[cnt_exp,'area'] = area_bucketed[tries][result]/100.0 #area_bucketed[tries][result]
                    panda.loc[cnt_exp,'path_len'] = filtered_path_df[tries][result]
                    cnt_exp += 1
            panda.norm_entropy = panda.norm_entropy.astype(float)
            panda.path_len = panda.path_len.astype(float)
            panda.area = panda.area.astype(float)
            
            sns.lineplot(data=panda, x='time', ci=None, y="area", ax=ax_map, color=color[cnt], label=i.split('/')[-1].replace('E1_','').replace('E2_','').replace('_S',''))
            sns.lineplot(data=panda, x='time', ci=None, y="path_len", ax=ax_map_split, color=color[cnt])

            ax_exp.plot(np.mean(filtered_path_df,axis=0), np.mean(area_bucketed, axis = 0)/100.0,color=color[cnt], label=i.split('/')[-1].replace('E1_','').replace('_S',''))
            ax_exp.plot(np.mean(filtered_path_df,axis=0), np.mean(entropy_bucketed, axis=0),color=color[cnt], zorder=101)
            ax_exp.set_ylim([0,1])
            sec_y.set_ylim([0,1])
            ax_exp.legend(ncol=2,loc='lower right')
            
#             ax_ent.legend()

            ax_map.set_ylim([0,1])
            ax_map.legend()

#             ax_area.legend()

            # plot CDF
#             edges = compute_bin_size(cov_vec)
#             n, bins, patches = ax1.hist(cov_vec, edges, cumulative=True, density=True, histtype='step', label=i.split('/')[-1], color=color[cnt])
#             patches[0].set_xy(patches[0].get_xy()[:-1])
#             ax1.plot(bins[1:], n,  label=columns[0],c=color[cnt] )

            cnt += 1
    
#     handles, labels = ax1.get_legend_handles_labels()
#     new_handles = [lines.Line2D([], [], c=h.get_edgecolor()) for h in handles]
#     ax1.legend(handles=new_handles, labels=labels)

#     if ate:
#         handles, labels = ax1.get_legend_handles_labels()
#         new_handles = [lines.Line2D([], [], c=h.get_edgecolor()) for h in handles]
#         ax1.legend(handles=new_handles, labels=labels)
#     fig1.suptitle('E1' if 'E1' in folder else 'E2')
#     fig1.show()
    
#     fig_map.suptitle('E1' if 'E1' in folder else 'E2')
    fig_map.show() 
#     fig_exp.suptitle('E1' if 'E1' in folder else 'E2')
    fig_exp.show()        
    
    separator = "-"
    if (saving_results):
        fig_map.savefig('map-{}-{}-{}.eps'.format('R',separator.join(columns),separator.join(tests)),format='eps',bbox_inches='tight')
        fig_exp.savefig('exp-{}-{}-{}.eps'.format('R',separator.join(columns),separator.join(tests)),format='eps',bbox_inches='tight')
    
#     fig_area.suptitle('E1' if 'E1' in folder else 'E2')
#     fig_area.show()        
    
    display(df)
    if (saving_results):
        print(df.to_latex(index=True, escape=False))

In [70]:
columns = ['S_']#,'S_O']#,'S_O','S_DW_O']#['S',]
tests = ['A','A_O','A_DW_O','OL_0','OL_2_3']#,'OL','I_OL','OL_FIX']#,'A_S','I_NF','NI','NI_NF',
folder = '/ps/project/irotate/PAPER-data/'
folder_E2 = os.path.join(folder,'R')

df_2_ = pd.DataFrame()
a = get_data(folder_E2, columns, tests, df_2_,0,20, True, True)

/ps/project/irotate/PAPER-data/R/S_A/0
/ps/project/irotate/PAPER-data/R/S_A/1
/ps/project/irotate/PAPER-data/R/S_A/2
/ps/project/irotate/PAPER-data/R/S_A/3
/ps/project/irotate/PAPER-data/R/S_A/4
/ps/project/irotate/PAPER-data/R/S_A/5
/ps/project/irotate/PAPER-data/R/S_A/6
/ps/project/irotate/PAPER-data/R/S_A/7
/ps/project/irotate/PAPER-data/R/S_A/8
/ps/project/irotate/PAPER-data/R/S_A/9
/ps/project/irotate/PAPER-data/R/S_A/10
/ps/project/irotate/PAPER-data/R/S_A/11
/ps/project/irotate/PAPER-data/R/S_A/12
/ps/project/irotate/PAPER-data/R/S_A/13
/ps/project/irotate/PAPER-data/R/S_A/14
/ps/project/irotate/PAPER-data/R/S_A/15
/ps/project/irotate/PAPER-data/R/S_A/16
/ps/project/irotate/PAPER-data/R/S_A/17
/ps/project/irotate/PAPER-data/R/S_A/18
/ps/project/irotate/PAPER-data/R/S_A/19
[3.6275232724993605, 1.63766858969415, 4.345512128231224, 0.8281301877825072, 0.7026357641983398]
/ps/project/irotate/PAPER-data/R/S_A_O/0
/ps/project/irotate/PAPER-data/R/S_A_O/1
/ps/project/irotate/PAPER-data

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


,S_A,S_A_O,S_A_DW_O,S_OL_0,S_OL_2_3
tot_area,mean: $83.257$ std: $13.239$,mean: $69.993$ std: $5.205$,mean: $73.344$ std: $1.371$,mean: $76.037$ std: $5.774$,mean: $72.924$ std: $18.419$
final_entropy_norm,mean: $0.283$ std: $0.054$,mean: $0.249$ std: $0.042$,mean: $0.235$ std: $0.015$,mean: $0.319$ std: $0.101$,mean: $0.266$ std: $0.041$
path length,mean: $21.201$ std: $3.094$,mean: $26.834$ std: $4.468$,mean: $28.751$ std: $4.666$,mean: $25.865$ std: $2.123$,mean: $23.855$ std: $2.722$
Loops per m,mean: $2.228$ std: $1.488$,mean: $3.852$ std: $1.434$,mean: $3.051$ std: $1.611$,mean: $2.397$ std: $1.057$,mean: $1.016$ std: $0.384$


\begin{tabular}{llllll}
\toprule
{} &                           S_A &                        S_A_O &                     S_A_DW_O &                       S_OL_0 &                      S_OL_2_3 \\
\midrule
tot_area           &  mean: $83.257$ std: $13.239$ &  mean: $69.993$ std: $5.205$ &  mean: $73.344$ std: $1.371$ &  mean: $76.037$ std: $5.774$ &  mean: $72.924$ std: $18.419$ \\
final_entropy_norm &    mean: $0.283$ std: $0.054$ &   mean: $0.249$ std: $0.042$ &   mean: $0.235$ std: $0.015$ &   mean: $0.319$ std: $0.101$ &    mean: $0.266$ std: $0.041$ \\
path length        &   mean: $21.201$ std: $3.094$ &  mean: $26.834$ std: $4.468$ &  mean: $28.751$ std: $4.666$ &  mean: $25.865$ std: $2.123$ &   mean: $23.855$ std: $2.722$ \\
Loops per m        &    mean: $2.228$ std: $1.488$ &   mean: $3.852$ std: $1.434$ &   mean: $3.051$ std: $1.611$ &   mean: $2.397$ std: $1.057$ &    mean: $1.016$ std: $0.384$ \\
\bottomrule
\end{tabular}

